In [0]:
##This block is only for access of files using google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#For accessing any file from google drive, first share it for public access. Copy its id from last part of its address. Then specify the two lines below.
downloaded = drive.CreateFile({'id':"11udsdq6pPymfbAE213Zfry14SgD0QOQl"})   # replace the id with id of file you want to access
downloaded.GetContentFile('spam.csv')        # replace the file name with your file


In [15]:
import pandas as pd
import numpy as np
import string

#import the data file
filename = 'spam.csv' 

df_sms = pd.read_csv('spam.csv',encoding='latin-1')
df_sms.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [16]:
#Remove the unwanted columns
df_sms = df_sms.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
df_sms = df_sms.rename(columns={"v1":"label", "v2":"sms"})
df_sms.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
#Print number of records
L = len(df_sms)
print(L)
#Example of accessing a column in pandas dataframe
df_sms.sms

5572


0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: sms, Length: 5572, dtype: object

In [0]:
#Define a Function to convert sms text to Lower case and remove stop words, punctuation and numbers

def preprocess_Text(input_Text):
  input_Text = input_Text.lower();
  stopwords = ['the','what','is','a','an','of', 'that']
  querywords = input_Text.split()

  resultwords  = [word for word in querywords if word not in stopwords]
  result = ' '.join(resultwords)

  exclude = set(string.punctuation)
  result = ''.join(ch for ch in result if ch not in exclude)

  exclude = set('0123456789')
  result = ''.join(ch for ch in result if ch not in exclude)

  return result;
  

In [19]:
#Test the preprocessing funtion
preprocess_Text('Hello, where4 is he.')

'hello where he'

In [0]:
#Preprocess all the sms texts
L = len(df_sms)
for i in range(0,L-1):
  df_sms['sms'][i] = preprocess_Text(df_sms['sms'][i])

In [21]:
#check if preprocessing was applied correctly by looking at any sms
df_sms['sms'][5]

'freemsg hey there darling its been  weeks now and no word back id like some fun you up for it still tb ok xxx std chgs to send å£ to rcv'

In [0]:
#Divide the dataframes into training and and testing set
from sklearn.utils import shuffle
df_sms = shuffle(df_sms)
training_Subset = df_sms.iloc[:round(len(df_sms)*0.9),:]   #90% data into training
test_Subset = df_sms.iloc[round(len(df_sms)*0.9):,:]       #10% data into testing
spam_Subset = training_Subset.query('label == "spam"')
ham_Subset = training_Subset.query('label == "ham"');

In [23]:
training_Subset['sms'][37]

KeyError: ignored

In [0]:
#combine all text into one large paragraph which shall be used to list unique words
L = len(training_Subset);
all_Text = ""
for i in training_Subset.index:
  all_Text = all_Text + " "+training_Subset['sms'][i];


In [24]:
all_Text

' gsoh good with spam ladiesu could b male gigolo  join uks fastest growing mens club reply oncall mjzgroup stop reply stop msgå£rcvd i want to send something can sell fast ltgt k not easy money  in cbe  in chennai nobody names their penis girls name this story doesnt add up at all thanks for temales it was wonderful thank have great week jordan got voted out last nite where one day training thanx but my birthday over already umma my life and vava umma love you lot dear oh ya got hip hop open haha i was thinking can go for jazz then zoom to cine actually tonight im free leh and theres kb lesson tonight ok me watching tv too fyi im taking quick shower be at epsilon in like ltgt min todays vodafone numbers ending with my last four digits are selected to received å£ award if your number matches please call  to claim your å£ award i thk ì gotta go home by urself cos ill b going out shopping  my frens present wot u up  u weirdo kent vale lor ìï wait  me there ar sms ac blind date u rodds m 

In [25]:
#make a table with all unique words
allWords = all_Text.split()

row_Names = [] 
for i in allWords:
      if not i in row_Names:
          row_Names.append(i);
print(row_Names)

['gsoh', 'good', 'with', 'spam', 'ladiesu', 'could', 'b', 'male', 'gigolo', 'join', 'uks', 'fastest', 'growing', 'mens', 'club', 'reply', 'oncall', 'mjzgroup', 'stop', 'msgå£rcvd', 'i', 'want', 'to', 'send', 'something', 'can', 'sell', 'fast', 'ltgt', 'k', 'not', 'easy', 'money', 'in', 'cbe', 'chennai', 'nobody', 'names', 'their', 'penis', 'girls', 'name', 'this', 'story', 'doesnt', 'add', 'up', 'at', 'all', 'thanks', 'for', 'temales', 'it', 'was', 'wonderful', 'thank', 'have', 'great', 'week', 'jordan', 'got', 'voted', 'out', 'last', 'nite', 'where', 'one', 'day', 'training', 'thanx', 'but', 'my', 'birthday', 'over', 'already', 'umma', 'life', 'and', 'vava', 'love', 'you', 'lot', 'dear', 'oh', 'ya', 'hip', 'hop', 'open', 'haha', 'thinking', 'go', 'jazz', 'then', 'zoom', 'cine', 'actually', 'tonight', 'im', 'free', 'leh', 'theres', 'kb', 'lesson', 'ok', 'me', 'watching', 'tv', 'too', 'fyi', 'taking', 'quick', 'shower', 'be', 'epsilon', 'like', 'min', 'todays', 'vodafone', 'numbers', 'e

In [0]:
#For each word find inspam probability and in-ham probability
word = '';
inSpamCount = 0;
inHamCount = 0;
columns = ['inSpamProbability','inHamProbability']
probability_Table = pd.DataFrame(index=row_Names, columns=columns)
for word in row_Names:
  inSpamCount = 0;
  inHamCount = 0;
  for i in spam_Subset['sms']:
    if(i.find(word)!=-1):
      inSpamCount = inSpamCount+1;
  for i in ham_Subset['sms']:
    if(i.find(word)!=-1):
      inHamCount = inHamCount+1;
  probability_Table.at[word, 'inSpamProbability'] = inSpamCount/len(spam_Subset);
  probability_Table.at[word,'inHamProbability'] = inHamCount/len(ham_Subset);


In [0]:
probability_Table.sort_values("inSpamProbability", axis = 0, ascending = False, 
                 inplace = True, na_position ='first') 

In [28]:
probability_Table

,inSpamProbability,inHamProbability
e,0.998527,0.946033
o,0.998527,0.94488
n,0.991163,0.905673
s,0.988218,0.824493
t,0.988218,0.912362
...,...,...
typical,0,0.000230627
happens,0,0.00161439
disasters,0,0.000230627
natural,0,0.000922509


In [0]:
#drop rows wherever spam or ham is zero probability
probability_Table = probability_Table[(probability_Table[['inSpamProbability','inHamProbability']] != 0).all(axis=1)]

In [30]:
probability_Table

,inSpamProbability,inHamProbability
e,0.998527,0.946033
o,0.998527,0.94488
n,0.991163,0.905673
s,0.988218,0.824493
t,0.988218,0.912362
...,...,...
return,0.00147275,0.00322878
avin,0.00147275,0.0142989
mens,0.00147275,0.000230627
party,0.00147275,0.00345941


#Assignment

1. Write a code to determine if a msg is spam or ham (2)
               def check_msg(test_Msg):
                ............
                ..........
2. Write a code to test all messages in test data and determine the following percentages (1)
  * True positive rate
  * True negative rate
  * False positive rate
  * False negative rate
  * Accuracy
  * Error_Rate